In [1]:
import ipsuite as ips

2023-04-13 08:54:44,217 (DEBUG): Welcome to IPS - the Interatomic Potential Suite!


In [5]:
mapping = ips.geometry.BarycenterMapping(data=None)

with ips.Project(automatic_node_names=True, remove_existing_graph=True) as project:
    mol = ips.configuration_generation.SmilesToAtoms(smiles="O")

    packmol = ips.configuration_generation.Packmol(
        data=[mol.atoms], count=[10], density=997
    )

    cp2k = ips.calculators.CP2KSinglePoint(
        data=packmol, cp2k_files=["GTH_BASIS_SETS", "GTH_POTENTIALS", "dftd3.dat"]
    )

    geopt = ips.calculators.ASEGeoOpt(
        calc=cp2k.calc,
        data=packmol.atoms,
        optimizer="BFGS",
        run_kwargs={"fmax": 0.1},
    )

    md = ips.calculators.ASEMD(
        calc=cp2k.calc,
        data=geopt.atoms,
        data_id=-1,
        temperature=300,
        time_step=1,
        friction=0.01,
        sampling_rate=1,
        dump_rate=1,
        steps=250,
    )  

    simulations = md.atoms

    for _ in range(5):
        md = ips.calculators.ASEMD(
            calc=cp2k.calc,
            data=md.atoms,
            data_id=-1,
            temperature=300,
            time_step=1,
            friction=0.01,
            sampling_rate=1,
            dump_rate=1,
            steps=250,
        )
        simulations += md.atoms

    e_hist = ips.analysis.EnergyHistogram(data=simulations, bins=100)
    f_hist = ips.analysis.ForcesHistogram(data=simulations)


project.run(repro=False)


Running DVC command: 'stage add --name SmilesToAtoms --force ...'
Running DVC command: 'stage add --name Packmol --force ...'
Running DVC command: 'stage add --name CP2KSinglePoint --force ...'
Running DVC command: 'stage add --name ASEGeoOpt --force ...'
Running DVC command: 'stage add --name ASEMD --force ...'
Running DVC command: 'stage add --name ASEMD_1 --force ...'
Running DVC command: 'stage add --name ASEMD_2 --force ...'
Running DVC command: 'stage add --name ASEMD_3 --force ...'
Running DVC command: 'stage add --name ASEMD_4 --force ...'
Running DVC command: 'stage add --name ASEMD_5 --force ...'
Running DVC command: 'stage add --name EnergyHistogram --force ...'
Running DVC command: 'stage add --name ForcesHistogram --force ...'
